In [28]:
import findspark
findspark.init()
findspark.find()

'/Users/djordjedimitrov/Documents/projects/python/DA642E-AI-and-Data-Management-for-IoT/LAB_5/TASK_1/venv/lib/python3.13/site-packages/pyspark'

In [29]:
import pyspark
findspark.find()

'/Users/djordjedimitrov/Documents/projects/python/DA642E-AI-and-Data-Management-for-IoT/LAB_5/TASK_1/venv/lib/python3.13/site-packages/pyspark'

In [30]:
from pyspark.sql import SparkSession

In [31]:
spark = SparkSession \
    .builder \
    .appName("Titanic Data") \
    .config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow") \
    .getOrCreate()

In [32]:
spark

In [33]:
df = (spark.read
      .format("csv")
        .option("header", "true")
        .load("titanic/train.csv"))

In [34]:
df.show(5)

+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 22|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 38|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 26|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 35|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 35|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+---+-----+-----+---------------

In [35]:
from pyspark.sql.functions import col

In [36]:
dataset = df.select(col("Survived").cast("float"),
                      col("Pclass").cast("float"),
                      col("Sex"),
                      col("Age").cast("float"),
                      col("Fare").cast("float"),
                      col("Embarked"))

dataset.show(4)
                            


+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
|     1.0|   1.0|female|35.0|   53.1|       S|
+--------+------+------+----+-------+--------+
only showing top 4 rows



In [37]:
from pyspark.sql.functions import isnull, when, count, col

In [38]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|177|   0|       2|
+--------+------+---+---+----+--------+



In [39]:
dataset = dataset.replace('?', None).dropna(how='any')

In [40]:
dataset.select([count(when(isnull(c), c)).alias(c) for c in dataset.columns]).show()

+--------+------+---+---+----+--------+
|Survived|Pclass|Sex|Age|Fare|Embarked|
+--------+------+---+---+----+--------+
|       0|     0|  0|  0|   0|       0|
+--------+------+---+---+----+--------+



In [41]:
dataset.show(3)

+--------+------+------+----+-------+--------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|
+--------+------+------+----+-------+--------+
|     0.0|   3.0|  male|22.0|   7.25|       S|
|     1.0|   1.0|female|38.0|71.2833|       C|
|     1.0|   3.0|female|26.0|  7.925|       S|
+--------+------+------+----+-------+--------+
only showing top 3 rows



In [42]:
from pyspark.ml.feature import StringIndexer

In [43]:
dataset = StringIndexer(
    inputCol='Sex',
    outputCol='Gender',
    handleInvalid='keep').fit(dataset).transform(dataset)

In [44]:
dataset = StringIndexer(
    inputCol='Embarked',
    outputCol='Boarded',
    handleInvalid='keep').fit(dataset).transform(dataset)

In [45]:
dataset.show(2)

+--------+------+------+----+-------+--------+------+-------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|Gender|Boarded|
+--------+------+------+----+-------+--------+------+-------+
|     0.0|   3.0|  male|22.0|   7.25|       S|   0.0|    0.0|
|     1.0|   1.0|female|38.0|71.2833|       C|   1.0|    1.0|
+--------+------+------+----+-------+--------+------+-------+
only showing top 2 rows



In [46]:
dataset = dataset.drop('Sex')
dataset = dataset.drop('Embarked')

In [47]:
from pyspark.ml.feature import VectorAssembler

In [48]:
require_featured = ['Pclass', 'Age', 'Fare', 'Gender', 'Boarded']
assember = VectorAssembler(inputCols=require_featured, outputCol='features')
transformed_data = assember.transform(dataset)

In [49]:
transformed_data.show(5)

+--------+------+----+-------+------+-------+--------------------+
|Survived|Pclass| Age|   Fare|Gender|Boarded|            features|
+--------+------+----+-------+------+-------+--------------------+
|     0.0|   3.0|22.0|   7.25|   0.0|    0.0|[3.0,22.0,7.25,0....|
|     1.0|   1.0|38.0|71.2833|   1.0|    1.0|[1.0,38.0,71.2833...|
|     1.0|   3.0|26.0|  7.925|   1.0|    0.0|[3.0,26.0,7.92500...|
|     1.0|   1.0|35.0|   53.1|   1.0|    0.0|[1.0,35.0,53.0999...|
|     0.0|   3.0|35.0|   8.05|   0.0|    0.0|[3.0,35.0,8.05000...|
+--------+------+----+-------+------+-------+--------------------+
only showing top 5 rows



In [50]:
(training_data, test_data) = transformed_data.randomSplit([0.8, 0.2])
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 576
Test Dataset Count: 136


In [51]:
from pyspark.ml.classification import RandomForestClassifier

In [52]:
rf = RandomForestClassifier(labelCol='Survived', featuresCol='features', maxDepth=5)
model = rf.fit(training_data)
predictions = model.transform(test_data)

In [53]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [54]:
evaluator = MulticlassClassificationEvaluator(labelCol='Survived', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)

Accuracy = 0.830882
